In [402]:
from pydub import AudioSegment
from python_speech_features import logfbank
from python_speech_features import ssc
from sklearn.cluster import KMeans

import numpy as np
import scipy.io.wavfile as wav
import speech_recognition as sr

In [30]:
def detect_leading_silence(sound, silence_threshold=-50.0, chunk_size=10):
    '''
    sound is a pydub.AudioSegment
    silence_threshold in dB
    chunk_size in ms

    iterate over chunks until you find the first one with sound
    '''
    trim_ms = 0 # ms

    assert chunk_size > 0 # to avoid infinite loop
    while sound[trim_ms:trim_ms+chunk_size].dBFS < silence_threshold and trim_ms < len(sound):
        trim_ms += chunk_size

    return trim_ms

In [60]:
def remover_silencio(comando):

    sound = AudioSegment.from_file(comando, format="wav")

    start_trim = detect_leading_silence(sound)
    end_trim = detect_leading_silence(sound.reverse())

    duration = len(sound)    
    trimmed_sound = sound[start_trim:duration-end_trim]
    
    comando_recortado = comando[:-4] + "_recortado.wav"

    trimmed_sound.set_channels(1).export(comando_recortado, format="wav")
    
    return (comando_recortado)

In [59]:
def limpar_audio(comando):

    r = sr.Recognizer()

    som = sr.AudioFile(comando)
    with som as source:
        r.adjust_for_ambient_noise(source)
        audio = r.record(source)
        
    comando_limpo = comando[:-4] + "_limpo.wav"

    with open(comando_limpo, "wb") as f:
        f.write(audio.get_wav_data())
        
    return (comando_limpo)

In [458]:
def extrair_caracteristica(audio):
    
    (rate,sig) = wav.read(remover_silencio(limpar_audio(audio)))
    fbank_feat = logfbank(sig,samplerate=rate, lowfreq=50, highfreq=3400)
#    fbank_feat = fbank_feat[0:,2:13]
    
    caracteristica = []
    for x in fbank_feat:
        for y in x:
            caracteristica.append(y)   
    
    return (caracteristica)

In [191]:
def dados(audios):
    caracteristicas = []
    
    for audio in audios:
        caracteristicas.append(extrair_caracteristica(audio))
        
    
    return (treinamento(caracteristicas))

In [441]:
def treinamento(comandos):
    menor_comando = len(comandos[0])
    
    for comando in comandos:
        print(len(comando))
        if (len(comando) < menor_comando):
            menor_comando = len(comando)
    x = 0        
    for comando in comandos:
        comandos[x] = comando[0:menor_comando]
        print(len(comandos[x]))
        x += 1

    X = np.array(comandos)

    kmeans = KMeans(precompute_distances=True, n_clusters=len(X), random_state=0, n_init=12, max_iter=3000, tol=0.00001, verbose=0, algorithm='elkan').fit(X)
    
    print (kmeans.cluster_centers_)
    
    print(kmeans.labels_)
    
    return (menor_comando)

In [459]:
dados(["0.wav","1.wav"])

5304
3328
3328
3328
[[-36.04365339   6.74189911   6.92031182 ...   7.31723658   7.12864289
    5.74549181]
 [-36.04365339   6.95822579   9.3185176  ...   6.32937626   5.7893004
    6.37928771]]
[0 1]


3328

In [407]:
type(extrair_caracteristica("0.wav"))

C:\ProgramData\Anaconda3\lib\site-packages\python_speech_features\base.py:111: RuntimeWarning: invalid value encountered in true_divide
  return numpy.dot(pspec*R,fb.T) / feat


list

In [195]:
len(extrair_caracteristica("3a.wav"))

4584

In [462]:
kmeans.predict([extrair_caracteristica("0.wav")[0:1944]])

array([1])

2232